# Titanic Kaggle Submission

This notebook encapsulates all the lessons from the previous two notebooks, creating a final submission for Kaggle. 

In [ ]:
import os
import sys
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTENC

# Modelling
# Standard Machine Learning Algorithms
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier
# Silect XGBoost
xgboost.config_context(verbosity=0)

# Neural Networks
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import InputLayer, Dense
from keras.callbacks import EarlyStopping

sys.path.append("../")
from utils import preprocessing_tools

print("Tensorflow version " + str(tf.__version__))

config = {
    'seed': 14,
    'balance_dataset': True,

    # NN Parameters
    'batch_size': 50,
    "no_epochs": 25
}

train_data = pd.read_csv('../input/train.csv')
train_data.head(5)